In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading the data from the CSV file

In [2]:
df=pd.read_csv('simmons2.csv')
df.head()

,Customer,HasCard,Spends,UsesCoupon
0,1,0,8.581598,1
1,2,0,1.464047,0
2,3,0,3.464725,0
3,4,1,0.801979,1
4,5,0,9.960163,1


#  Dividing the DataFrame into features and target variables

In [3]:
A=df[['Spends','HasCard']]
A.head()

,Spends,HasCard
0,8.581598,0
1,1.464047,0
2,3.464725,0
3,0.801979,1
4,9.960163,0


# A represents the feature variables

In [4]:
A = A.to_numpy()


In [5]:
y= df[['UsesCoupon']]
y.head()

,UsesCoupon
0,1
1,0
2,0
3,1
4,1


# y represents the target variable

In [6]:
y = y.to_numpy()
y[:5,]

array([[1],
       [0],
       [0],
       [1],
       [1]], dtype=int64)

# Question 1.b) 

In [7]:
df=df.set_index('Customer')
X=df.drop(['UsesCoupon'],axis=1)

y=df['UsesCoupon']
y=np.array(y)

# Spliting the data into test and train.

In [8]:
X_train = X[:500]
X_test = X[500:]
y_train = y[:500]
y_test = y[500:]

# Sigmoid function is used to map the output between 0-1
# Loss calculates the cost function
# Fit calculates the gradient and updates the theta
# Predict is use to predict class of test data

In [9]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=1000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    # Activator to map the value between 0-1
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    # Cost function
    def __loss(self, h, y):
        return -1/len(y)*((-y * np.log(h) - (1 - y) * np.log(1 - h)))
    
    # Gradient descent
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            loss = self.__loss(h, y)
                
            if(self.verbose ==True and i % self.num_iter == 0):
                print(f'loss: {loss} \t')
                
    # value between 0-1 i.e probabilities
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
            
            s=np.dot(X, self.theta)
        return self.__sigmoid(s)
    
    
    def predict(self, X):
        return self.predict_prob(X).round()

# Initialize class as model

In [10]:
model = LogisticRegression(lr=0.2, num_iter=300)

# Running fit method. This performs gradient descent and the theta values are updated.  

In [11]:
model.fit(X_train, y_train)

 # Predicted probablity for the X matrix.

In [12]:
probabilities = model.predict_prob(X)
probabilities[:20]

array([0.97556159, 0.2761175 , 0.5850295 , 0.80423983, 0.98992549,
       0.99769625, 0.99629   , 0.9803072 , 0.97238634, 0.97990903,
       0.55343591, 0.83893234, 0.98852246, 0.9884719 , 0.35129839,
       0.78808529, 0.99719585, 0.9342712 , 0.83613243, 0.76432362])

# Weights for the given problem.

In [13]:
model.theta

array([-1.92042174,  2.80940999,  0.65340729])

# With the use of theta values, predict the target values of X_test. (i.e y_test) 

In [14]:
predicted = model.predict(X_test)

In [15]:
t_pred=0 #true predicted
f_pred=0 #false predicted
for i,j in zip(predicted,y_test):
    if i == j:
        t_pred +=1
    else:
        f_pred +=1
    

In [16]:
Accuracy = (t_pred/(t_pred+f_pred))*100
print('ACCURACY ON THE TEST SET IS {} %'.format(Accuracy))

ACCURACY ON THE TEST SET IS 94.0 %


### Let x = [x_1; x_2; x_3]T be the minimum of f you computed from the previous question. The vector x is the coeficients you need for your predictive model. In other words, you can predict the probability that a customer uses a coupon based on the formula ^y = (x_1+ x_2 Spends + x_3 HasCard) if you know the spending level of a customer or whether the customer has a card. For example, if the customer is spending 1000 dollars (then the variable Spends = 1) and he has a store card; your prediction will be ^y = (x_1+ x_2 (1) + x_3 (1)):
# Does having a store card increase the likelihood of the customers' using the coupons?  

In [17]:
dollars=[x for x in range(1,10,1)]

# has_discount represents spending with discount coupon. no_discount represents spending without discount coupon. 

In [18]:
has_discount=pd.DataFrame(dollars,np.ones(9,dtype=int)).reset_index()
has_discount

,index,0
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
5,1,6
6,1,7
7,1,8
8,1,9


In [19]:
no_discount=pd.DataFrame(dollars,np.zeros(9,dtype=int)).reset_index()
no_discount

,index,0
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
5,0,6
6,0,7
7,0,8
8,0,9


# Probability of spend having a discount and not have a discount

In [20]:
a_cust=model.predict_prob(has_discount)
b_cust=model.predict_prob(no_discount)

In [21]:
new=pd.DataFrame(a_cust,b_cust).reset_index()
new['Spends']=dollars
new=new.set_index('Spends')
new=new.T
new = new.rename(index={'index': 'No', 0:'Yes'})

In [22]:
new

Spends,1,2,3,4,5,6,7,8,9
No,0.219769,0.351237,0.509949,0.666680,0.793576,0.880799,0.934222,0.964663,0.981298
Yes,0.823813,0.899872,0.945278,0.970762,0.984572,0.991913,0.995776,0.997798,0.998853


#  As a store manager my strategy would be to give out coupons to every customer beacuse the probabilty of customer spending becomes high if they have a discount coupon